<a href="https://colab.research.google.com/github/dongjaeseo/CS2705/blob/main/lung_cancer_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
class mask_image():
    def __init__(self, image):
        self.image = image
        self.boundary = self.find_boundary()
        self.cg = self.centroid()
        self.theta_dict = self.theta()

    def centroid(self):
        row_sum = 0
        column_sum = 0
        count = 0
        for i, row in enumerate(self.image):
            for j, pixel in enumerate(row):
                if pixel == 1:
                    row_sum += i
                    column_sum += j
                    count += 1

        if count == 0:
            raise TypeError("Wrong image input: Only masked image of 0 and 1 is allowed")

        return column_sum/count, row_sum/count

    def find_boundary(self):
        boundary = []
        for i, row in enumerate(self.image):
            for j, pixel in enumerate(row):

                if i != 0:
                    if self.image[i-1][j] < pixel:
                        boundary.append((j,i-0.5))
                
                if j != len(self.image[0])-1:
                    if self.image[i][j+1] < pixel:
                        boundary.append((j+0.5, i))
                
                if i != len(self.image)-1:
                    if self.image[i+1][j] < pixel:
                        boundary.append((j,i+0.5))

                if j != 0:
                    if self.image[i][j-1] < pixel:
                        boundary.append((j-0.5, i))

        return boundary
    
    def theta(self):
        theta_boundary = {}
        for bound in self.boundary:
            theta = np.arctan(self.cg[0]-bound[0])/(bound[1]-self.cg[1])
            # in 3rd and 4th quadrant // arctan (-pi, pi)
            if bound[1]-self.cg[1] < 0:
                theta += np.pi
            theta_boundary.update({theta: bound})
        
        theta_boundary = dict(sorted(theta_boundary.items(), key=lambda item: item[0]))

        return theta_boundary
    
    def find_dist(self):
        distance = []
        for bound in self.boundary:
            dist = np.sqrt(np.square(bound[0]-self.cg[0]) + np.square(bound[1]-self.cg[1]))
            distance.append(dist)
        
        return distance

    def plot(self):
        plt.imshow(self.image, cmap = 'gray')
        for i in self.boundary:
            plt.plot(i[0], i[1], marker = "o", markersize = 3, markeredgecolor = "red", markerfacecolor = "green")
            plt.plot(self.cg[0], self.cg[1], marker = "o", markersize = 1, markeredgecolor = "red", markerfacecolor = "green")
        plt.show()

In [ ]:
image1 = cv2.imread('/content/drive/MyDrive/project/mask2.png', 0)
image1 = image1/ 255.
image1 = np.where(image1 < 0.5, 0, 1)

image1 = mask_image(image1)
distance = image1.find_dist()
print('distance: ')
print(distance)

theta_dict = image1.theta()
print('\ntheta - coordinate:')
print(theta_dict)

image1.plot()